# Web Scraping con Beautiful Soup

* * * 

### Iconos utilizados en este notebook
🔔 **Preguntas**: Una pregunta rápida para ayudarte a entender qué está pasando.<br>
🥊 **Desafío**: Ejercicio interactivo. Lo trabajaremos en el taller.!<br>
⚠️ **Advertencia**: Aviso sobre cuestiones complicadas o errores comunes.<br>
💡 **Tip**:Cómo hacer algo de forma un poco más eficiente o efectiva.<br>
🎬 **Demo**: Mostrando algo más avanzado: ¡para que sepas para qué se puede usar Python!<br>

### Objetivos de aprendizaje
1. [Objetivos de aprendizaje](#when)
2. [Extracción y análisis de HTML](#extract)
3. [Desmantelando la Asamblea General de Illinois](#scrape)

<a id='when'></a>

# “¿Hacer scraping o no hacerlo?”

Cuando queremos acceder a datos de la web, primero debemos asegurarnos de que el sitio web que nos interesa ofrezca una API web. Plataformas como Twitter, Reddit y The New York Times ofrecen API. **Echa un vistazo a D-Lab [Python Web APIs](https://github.com/dlab-berkeley/Python-Web-APIs) Taller si quieres aprender a utilizar las API.**

Sin embargo, a menudo no existe una API web. En estos casos, podemos recurrir al web scraping, donde extraemos el HTML subyacente de una página web y obtenemos directamente la información deseada. Existen varios paquetes en Python que podemos usar para realizar estas tareas. Nos centraremos en dos paquetes: Requests y Beautiful Soup.

Nuestro estudio de caso consistirá en extraer información sobre el [Senadores estatales de Illinois](http://www.ilga.gov/senate), así como el [lista de facturas](http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True) Cada senador ha patrocinado. Antes de empezar, revise estos sitios web para conocer su estructura.

## Instalación

Utilizaremos dos paquetes principales: [Solicitudes](http://docs.python-requests.org/en/latest/user/quickstart/) y [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Continúe e instale estos paquetes, si aún no lo ha hecho:

In [1]:
# 🌐 La librería requests es necesaria para hacer solicitudes HTTP y descargar páginas web.
# 🕸️ Esto es fundamental para hacer web scraping (extraer información de páginas web).
%pip install requests  

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 🥣 La instrucción %pip install beautifulsoup4 sirve para instalar la librería Beautiful Soup 4 en tu entorno de Jupyter Notebook.
# 🕸️ Beautiful Soup es esencial para analizar y extraer información de archivos HTML y XML, lo que facilita el web scraping.
%pip install beautifulsoup4


   ------------- -------------------------- 1/3 [soupsieve]
   -------------------------- ------------- 2/3 [beautifulsoup4]
   -------------------------- ------------- 2/3 [beautifulsoup4]
   -------------------------- ------------- 2/3 [beautifulsoup4]
   ---------------------------------------- 3/3 [beautifulsoup4]

Note: you may need to restart the kernel to use updated packages.


También instalaremos el paquete `lxml`, que ayuda a soportar parte del análisis que realiza Beautiful Soup:

In [3]:
%pip install lxml

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 15.9 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

<a id='extract'></a>

# Extracción y análisis de HTML

Para extraer y analizar HTML correctamente, seguiremos los siguientes 4 pasos:
1. Realizar una solicitud GET
2. Analizar la página con Beautiful Soup
3. Buscar elementos HTML
4. Obtener los atributos y el texto de estos elementos

## Paso 1: Realizar una solicitud GET para obtener el HTML de una página

Podemos usar la biblioteca Requests para:

1. Realizar una solicitud GET a la página y
2. Leer el código HTML de la página web.

El proceso de realizar una solicitud y obtener un resultado es similar al del flujo de trabajo de la API web. Sin embargo, ahora realizamos una solicitud directamente al sitio web y tendremos que analizar el HTML nosotros mismos. Esto contrasta con recibir datos organizados en una salida JSON o XML más sencilla.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# View some output
print(src[:1000])

## Paso 2: Analizar la página con Beautiful Soup

Ahora, usamos la función `BeautifulSoup` para analizar la respuesta en un árbol HTML. Esto devuelve un objeto (llamado **objeto soup**) que contiene todo el HTML del documento original.

Si se produce un error relacionado con una biblioteca de análisis, asegúrese de haber instalado el paquete `lxml` para proporcionar a Beautiful Soup las herramientas de análisis necesarias.

In [ ]:
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')
# Take a look
print(soup.prettify()[:1000])

La salida se ve bastante similar a la anterior, pero ahora está organizada en un objeto 'soup' que nos permite recorrer la página más fácilmente.

## Paso 3: Buscar elementos HTML

Beautiful Soup cuenta con varias funciones para encontrar componentes útiles en una página. Beautiful Soup permite encontrar elementos por:

1. Etiquetas HTML
2. Atributos HTML
3. Selectores CSS

Primero, busquemos **etiquetas HTML**.

La función `find_all` busca en el árbol `soup` todos los elementos con una etiqueta HTML específica y los devuelve.

¿Qué hace el siguiente ejemplo?

In [ ]:
# Find all elements with a certain tag
a_tags = soup.find_all("a")
print(a_tags[:10])

Dado que `find_all()` es el método más popular en la API de búsqueda de Beautiful Soup, puedes usar un atajo. Si tratas el objeto BeautifulSoup como si fuera una función, es lo mismo que llamar a `find_all()` en ese objeto.

Estas dos líneas de código son equivalentes:

In [ ]:
a_tags = soup.find_all("a")
a_tags_alt = soup("a")
print(a_tags[0])
print(a_tags_alt[0])

¿Cuántos enlaces obtuvimos?

In [ ]:
print(len(a_tags))

¡Eso es muchísimo! Muchos elementos de una página tendrán la misma etiqueta HTML. Por ejemplo, si buscas todo con la etiqueta `a`, probablemente obtendrás más resultados, muchos de los cuales quizás no quieras. Recuerda que la etiqueta `a` define un hipervínculo, por lo que normalmente encontrarás muchos en cualquier página.

¿Qué sucedería si quisiéramos buscar etiquetas HTML con ciertos atributos, como clases CSS específicas?

Podemos hacerlo añadiendo un argumento adicional a `find_all`. En el siguiente ejemplo, buscamos todas las etiquetas `a` y luego las filtramos con `class_="sidemenu"`.

In [ ]:
# Get only the 'a' tags in 'sidemenu' class
side_menus = soup("a", class_="sidemenu")
side_menus[:5]

Una forma más eficiente de buscar elementos en un sitio web es mediante un selector CSS. Para ello, debemos usar un método diferente llamado `select()`. Simplemente pase una cadena a `.select()` para obtener todos los elementos con esa cadena como un selector CSS válido.

En el ejemplo anterior, podemos usar `"a.sidemenu"` como selector CSS, que devuelve todas las etiquetas `a` con la clase `sidemenu`.

In [ ]:
# Get elements with "a.sidemenu" CSS Selector.
selected = soup.select("a.sidemenu")
selected[:5]

## 🥊Desafío: Encontrar todo

Usa BeautifulSoup para encontrar todos los elementos `a` con la clase `mainmenu`.

In [ ]:
# YOUR CODE HERE


Paso 4: Obtener los atributos y el texto de los elementos

Una vez identificados los elementos, necesitamos la información de acceso de cada uno. Normalmente, esto implica dos cosas:

1. Texto
2. Atributos

Obtener el texto dentro de un elemento es sencillo. Solo tenemos que usar el miembro `text` de un objeto `tag`:

In [ ]:
# Get all sidemenu links as a list
side_menu_links = soup.select("a.sidemenu")

# Examine the first link
first_link = side_menu_links[0]
print(first_link)

# What class is this variable?
print('Class: ', type(first_link))

¡Es una etiqueta de Beautiful Soup! Esto significa que tiene un miembro "texto":

In [ ]:
print(first_link.text)

A veces necesitamos el valor de ciertos atributos. Esto es especialmente relevante para las etiquetas «a» o enlaces, donde el atributo «href» nos indica adónde lleva el enlace.

💡 **Consejo**: Puedes acceder a los atributos de una etiqueta tratándola como un diccionario:

In [ ]:
print(first_link['href'])

## 🥊 Desafío: Extraer atributos específicos

Extraer todos los atributos `href` de cada URL `mainmenu`.

In [ ]:
# YOUR CODE HERE


<a id='scrape'></a>

# Análisis de la Asamblea General de Illinois

Aunque parezca increíble, estas son las herramientas fundamentales para analizar un sitio web. Una vez que dediques más tiempo a familiarizarte con HTML y CSS, simplemente será cuestión de comprender la estructura de un sitio web específico y aplicar inteligentemente las herramientas de Beautiful Soup y Python.

Apliquemos estas habilidades para analizar la [98.ª Asamblea General de Illinois](http://www.ilga.gov/senate/default.asp?GA=98).

En concreto, nuestro objetivo es analizar la información de cada senador, incluyendo su nombre, distrito y partido.

## Rastrear y analizar la página web

Rastreemos y analicemos la página web con las herramientas que aprendimos en la sección anterior.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")

## Buscar los elementos de la tabla

Nuestro objetivo es obtener los elementos de la tabla en la página web. Recuerde: las filas se identifican con la etiqueta `tr`. Usemos `find_all` para obtener estos elementos.

In [ ]:
# Get all table row elements
rows = soup.find_all("tr")
len(rows)

⚠️ **Advertencia**: Ten en cuenta que `find_all` obtiene *todos* los elementos con la etiqueta `tr`. Solo necesitamos algunos. Si usamos la función "Inspeccionar" de Google Chrome y observamos con atención, podemos usar selectores CSS para obtener solo las filas que nos interesan. En concreto, queremos las filas internas de la tabla:

In [ ]:
# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')

for row in rows[:5]:
    print(row, '\n')

Parece que queremos todo lo que queda después de las dos primeras filas. Empecemos con una sola fila y construyamos nuestro bucle a partir de ahí.

In [ ]:
example_row = rows[2]
print(example_row.prettify())

Desglosemos esta fila en sus celdas/columnas mediante el método `select` con selectores CSS. Si analizamos el HTML con atención, hay un par de maneras de hacerlo.

* Podríamos identificar las celdas por su etiqueta `td`.
* Podríamos usar el nombre de clase `.detail`.
* Podríamos combinar ambos y usar el selector `td.detail`.

In [ ]:
for cell in example_row.select('td'):
    print(cell)
print()

for cell in example_row.select('.detail'):
    print(cell)
print()

for cell in example_row.select('td.detail'):
    print(cell)
print()

Podemos confirmar que todos son iguales.

In [ ]:
assert example_row.select('td') == example_row.select('.detail') == example_row.select('td.detail')

Utilicemos el selector `td.detail` para ser lo más específicos posible.

In [ ]:
# Select only those 'td' tags with class 'detail' 
detail_cells = example_row.select('td.detail')
detail_cells

La mayoría de las veces, nos interesa el **texto** real de un sitio web, no sus etiquetas. Recordemos que para obtener el texto de un elemento HTML, usamos el miembro `text`:

In [ ]:
# Keep only the text in each of those cells
row_data = [cell.text for cell in detail_cells]

print(row_data)

¡Se ve bien! Ahora solo necesitamos usar nuestros conocimientos básicos de Python para obtener los elementos de esta lista que necesitamos. Recuerda: queremos el nombre del senador, su distrito y su partido.

In [ ]:
print(row_data[0]) # Name
print(row_data[3]) # District
print(row_data[4]) # Party

## Eliminando filas basura

Vimos al principio que no todas las filas que obtuvimos corresponden a un senador. Tendremos que hacer limpieza antes de continuar. Vean algunos ejemplos:

In [ ]:
print('Row 0:\n', rows[0], '\n')
print('Row 1:\n', rows[1], '\n')
print('Last Row:\n', rows[-1])

Al escribir nuestro bucle for, queremos que solo se aplique a las filas relevantes. Por lo tanto, debemos filtrar las filas irrelevantes. Para ello, comparamos algunas de estas filas con las que necesitamos, observamos sus diferencias y luego formulamos esto en una condición.

Como puedes imaginar, hay muchas maneras de hacerlo, y dependerá del sitio web. Aquí te mostraremos algunas para que te hagas una idea de cómo hacerlo.

In [ ]:
# Bad rows
print(len(rows[0]))
print(len(rows[1]))

# Good rows
print(len(rows[2]))
print(len(rows[3]))

Quizás las buenas filas tengan una longitud de 5. Comprobémoslo:

In [ ]:
good_rows = [row for row in rows if len(row) == 5]

# Let's check some rows
print(good_rows[0], '\n')
print(good_rows[-2], '\n')
print(good_rows[-1])

Encontramos una fila de pie de página en nuestra lista que queremos evitar. Probemos algo diferente:

In [ ]:
rows[2].select('td.detail') 

In [ ]:
# Bad row
print(rows[-1].select('td.detail'), '\n')

# Good row
print(rows[5].select('td.detail'), '\n')

# How about this?
good_rows = [row for row in rows if row.select('td.detail')]

print("Checking rows...\n")
print(good_rows[0], '\n')
print(good_rows[-1])

¡Parece que encontramos algo que funcionó!

## Unir todo en un bucle

Ahora que hemos visto cómo obtener los datos que queremos de una fila y filtrar las filas que no necesitamos, vamos a unirlo todo en un bucle.

In [ ]:
# Define storage list
members = []

# Get rid of junk rows
valid_rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in valid_rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    # Store in a tuple
    senator = (name, district, party)
    # Append to list
    members.append(senator)

In [ ]:
# Should be 61
len(members)

Echemos un vistazo a lo que tenemos en "miembros".

In [ ]:
print(members[:5])

## 🥊  Desafío: Obtener elementos `href` que apunten a los proyectos de ley de los miembros

El código anterior recupera información sobre:

- el nombre del senador,
- su número de distrito,
- y su partido.

Ahora queremos recuperar la URL de la lista de proyectos de ley de cada senador. Cada URL seguirá un formato específico.

El formato de la lista de proyectos de ley de un senador determinado es:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

para obtener algo como:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

donde `MEMBER_ID=1911`.

Deberías poder ver que, lamentablemente, `MEMBER_ID` no se extrae actualmente en nuestro código de extracción.

Tu tarea inicial es modificar el código anterior para que también **recuperemos la URL completa que apunta a la página correspondiente de los proyectos de ley patrocinados por las primarias** de cada miembro, y la devolvamos junto con su nombre, distrito y partido.

Consejos:

* Para ello, deberá obtener el elemento de anclaje correspondiente (`<a>`) en la fila de cada legislador de la tabla. Puede usar el método `.select()` en el objeto `row` del bucle para hacerlo, de forma similar al comando que busca todas las celdas `td.detail` de la fila. Recuerde que solo necesitamos el enlace a los proyectos de ley del legislador, no a los comités ni a su página de perfil.
* El HTML de los elementos de anclaje se verá como `<a href="/senate/Senator.asp/...">Proyectos de ley</a>`. La cadena del atributo `href` contiene el enlace **relativo** que buscamos. Puede acceder a un atributo de un objeto `Tag` de BeatifulSoup de la misma manera que accede a un diccionario de Python: `anchor['attributeName']`. Consulta la <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentación</a> para obtener más detalles.
* Hay muchas maneras diferentes de usar BeautifulSoup. Puedes usar cualquier método para extraer el `href`.

El código se ha completado parcialmente. Complétalo donde dice `#TU CÓDIGO AQUÍ`. Guarda la ruta en un objeto llamado `full_path`.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Create empty list to store our data
members = []

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
# Get rid of junk rows
rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail') 
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]

    # YOUR CODE HERE
    full_path = ''

    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

In [ ]:
# Uncomment to test 
# members[:5]

## 🥊  Desafío: Modulariza tu código

Convierte el código anterior en una función que acepte una URL, rastree la URL para encontrar sus senadores y devuelva una lista de tuplas con información sobre cada senador. 

In [ ]:
# YOUR CODE HERE
def get_members(url):
    return [___]


In [ ]:
# Test your code
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = get_members(url)
len(senate_members)

## 🥊 Desafío práctico: Escribir una función de scraping

Queremos scraping las páginas web correspondientes a los proyectos de ley patrocinados por cada proyecto de ley.

Escribir una función llamada `get_bills(url)` para analizar la URL de un proyecto de ley. Esto implica:

- Solicitar la URL mediante la biblioteca <a href="http://docs.python-requests.org/en/latest/">`requests`</a>
- Usar las funciones de la biblioteca `BeautifulSoup` para encontrar todos los elementos `<td>` con la clase `billlist`
- Devolver una _lista_ de tuplas, cada una con:
- Descripción (2.ª columna)
- Cámara (S o H) (3.ª columna)
- La última acción (4.ª columna)
- La fecha de la última acción (5.ª columna)

Esta función se ha completado parcialmente. Complete el resto.

In [ ]:
def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src)
    rows = soup.select('tr')
    bills = []
    for row in rows:
        # YOUR CODE HERE
        #bill_id =
        #description =
        #chamber =
        #last_action =
        #last_action_date =
        bill = (bill_id, description, chamber, last_action, last_action_date)
        bills.append(bill)
    return bills

In [ ]:
# Uncomment to test your code
# test_url = senate_members[0][3]
# get_bills(test_url)[0:5]

### Extraer todos los proyectos de ley

Finalmente, cree un diccionario `bills_dict` que asigne un número de distrito (la clave) a una lista de proyectos de ley (el valor) provenientes de ese distrito. Puede hacerlo recorriendo en bucle todos los miembros del senado en `members_dict` y llamando a `get_bills()` para cada una de las URL de sus proyectos de ley asociados.

**NOTA:** Por favor, llame a la función `time.sleep(1)` en cada iteración del bucle para no destruir el sitio web del estado.

In [ ]:
# YOUR CODE HERE


In [ ]:
# Uncomment to test your code
# bills_dict[52]